# Duck Typing

## Example 1: `UnitRange`

In [45]:
x = 1:30

1:30

In [104]:
typeof(x)

UnitRange{Int64}

In [105]:
typeof(x) <: AbstractArray

true

Because it is a subtype of `AbstractArray` we can do array-like things with it (it should basically behave like an array!)

In [106]:
x[3]

3

In [107]:
size(x)

(30,)

In [108]:
eltype(x)

Int64

However, it's not implemented like a regular `Array` at all.

In fact, it's just two numbers! We can see this by looking at it's fields:

In [109]:
fieldnames(typeof(x))

(:start, :stop)

or just by inspecting the source code

In [110]:
@which UnitRange(1, 30)

UnitRange(start::T, stop::T) where T<:Real in Base at range.jl:396

It is an `immutable` type which just holds the start and stop values.

This means that indexing, `A[i]`, is not just a look-up but a (small) function (try `@which getindex(x, 4)`).

What's nice about this is that we can use it in calculations and no array, containing the numbers from 1 to 30, is ever created.

Allocating memory is typically costly.

In [111]:
@time collect(1:10000000);

  0.062068 seconds (2 allocations: 76.294 MiB, 6.24% gc time)


But creating an immutable type of two numbers is essentially free, no matter what those two numbers are:

In [112]:
@time 1:10000000;

  0.000001 seconds


Yet, in code they **behave** in the same way.

### Example 2: Diagonal matrix

Let's create a simple custom `DiagonalMatrix` type that can represent square diagonal matrices, i.e.

$$ D = \left( \begin{matrix} x & 0 & 0 & 0 \\ 0 & y & 0 & 0 \\ 0 & 0 & z & 0 \\ 0 & 0 & 0 & \ddots \end{matrix} \right) $$

In [1]:
struct DiagonalMatrix{T} <: AbstractArray{T,2}
    diag::Vector{T}
end

In the spirit of duck typing, we integrate our `DiagonalMatrix` into Julia's type hierarchy by making it a subtype (`<:`) of `AbstractMatrix` to indicate **array-like behavior**. (Note that this does not indicate inheritence of structure!)

Of course, to actually make it behave like a matrix (a two-dimensional array) we must also implement (parts of) the [`AbstractArray` interface](https://docs.julialang.org/en/v1/manual/interfaces/#man-interface-array-1).

In [59]:
# implement AbstractArray interface
function Base.getindex(D::DiagonalMatrix, i::Int, j::Int)
    if i == j
        return D.diag[i]
    else
        return zero(eltype(D))
    end
end

function Base.setindex!(D::DiagonalMatrix, v, i::Int, j::Int)
    if i == j
        D.diag[i] = v
    else
        throw(ArgumentError("cannot set off-diagonal entry ($i, $j)"))
    end
    return v
end

Base.size(D::DiagonalMatrix) = (length(D.diag), length(D.diag))

In [68]:
D = DiagonalMatrix([1,2,3])

3×3 DiagonalMatrix{Int64}:
 1  0  0
 0  2  0
 0  0  3

Note how it's automagically pretty printed (despite the fact that we never defined any special printing)!

In [69]:
D[2,2]

2

In [70]:
D[1,2]

0

In [71]:
size(D)

(3, 3)

In [75]:
D[3,3] = 5

5

In [78]:
D

3×3 DiagonalMatrix{Int64}:
 1  0  0
 0  2  0
 0  0  5

But that's not it. Because of duck typing, all kinds of different functions now "just work".

In [7]:
eltype(D) # element data type

Int64

In [8]:
D + D # addition

3×3 Matrix{Int64}:
 2  0  0
 0  4  0
 0  0  6

In [9]:
D * D # multiplication

3×3 Matrix{Int64}:
 1  0  0
 0  4  0
 0  0  9

In [10]:
inv(D) # inversion

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  0.5  0.0
 0.0  0.0  0.333333

In [11]:
sin.(D) # broadcasting

3×3 Matrix{Float64}:
 0.841471  0.0       0.0
 0.0       0.909297  0.0
 0.0       0.0       0.14112

In [12]:
using LinearAlgebra
eigen(D) # eigensolver

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 1.0
 2.0
 3.0
vectors:
3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

Of course, so far, these operations have suboptimal performance because they don't utilize the special structure inherent to our `DiagonalMatrix` but fall back to generic implementations.

In [14]:
@which D + D

+(A::AbstractArray, B::AbstractArray) in Base at arraymath.jl:6

In [22]:
Base.:+(Da::DiagonalMatrix, Db::DiagonalMatrix) = DiagonalMatrix(Da.diag + Db.diag)

In [23]:
@which D + D

+(Da::DiagonalMatrix, Db::DiagonalMatrix) in Main at In[22]:1

Important note: **user defined types are just as good as built-in types!**

There is nothing special about built-in types. In fact, [they are implemented in essentially the same way](https://github.com/JuliaLang/julia/blob/master/stdlib/LinearAlgebra/src/diagonal.jl#L5)!

Let us quickly confirm that our `DiagonalMatrix` type does not come with any performance overhead by benchmarking it in a simple function.

## Benchmarking with [`BenchmarkTools.jl`](https://github.com/JuliaCI/BenchmarkTools.jl)

Benchmarking is difficult to do right for many reasons
* computers are noisy machines
* global vs local scope
* the first function call is special in Julia (more later)
* ...

In [121]:
g(x) = x + 2*x

g (generic function with 1 method)

In [124]:
x = rand(2,2)
@time g.(x)

  0.000048 seconds (3 allocations: 128 bytes)


2×2 Matrix{Float64}:
 2.49731  1.81634
 1.42723  1.19451

In [125]:
function f()
    x = rand(2,2)
    @time g.(x)
end

f (generic function with 1 method)

In [126]:
f()

  0.000001 seconds (1 allocation: 96 bytes)


2×2 Matrix{Float64}:
 0.542584  1.83312
 1.3948    1.59456

Fortunately, there are tools that do this for us. In addition, they also collect some statistics by running the benchmark multiple times.

General rule: **Don't use `@time` but `@btime`** from [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) and interpolate (`$`) input arguments.

In [132]:
using BenchmarkTools

In [133]:
@btime g.($x)

  23.845 ns (1 allocation: 96 bytes)


2×2 Matrix{Float64}:
 2.49731  1.81634
 1.42723  1.19451

In [135]:
@benchmark g.($x)

BenchmarkTools.Trial: 10000 samples with 996 evaluations.
 Range (min … max):  23.762 ns … 546.436 ns  ┊ GC (min … max): 0.00% … 92.51%
 Time  (median):     24.515 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   25.972 ns ±  20.556 ns  ┊ GC (mean ± σ):  3.40% ±  4.10%

  ▂▇█▅▂        ▂▂▁                                             ▂
  ██████▇▇▆▆▆▇██████▇▅▄▄▄▄▁▁▄▄▁▁▁▁▃▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▆▇█ █
  23.8 ns       Histogram: log(frequency) by time      44.7 ns <

 Memory estimate: 96 bytes, allocs estimate: 1.

### Custom types are just as good as built-in types

Let's compare our custom `DiagonalMatrix` against the standard `Diagonal` type that ships in the `LinearAlgebra` standard library.

In [140]:
using LinearAlgebra
x = rand(100);
Djl = Diagonal(x)
D = DiagonalMatrix(x)
@btime $Djl + $Djl;
@btime $D + $D;

  42.128 ns (1 allocation: 896 bytes)
  41.667 ns (1 allocation: 896 bytes)


# Core messages of this notebook

* Duck typing is about **shared behavior** instead of shared structure.
* **User defined types are as good as built-in types.**
* We can **extend Base functions** for our types to implement arithmetics and such.
* **Subtyping an existing interface** can give lots of functionality for free.
* Functions should almost always be benchmarked with **BenchmarkTools.jl's `@btime` and `@benchmark`** instead of `@time`.